## Experts Answers Data Integration
---

In [1]:
import pandas as pd

In [2]:
tables = []
sheets = ['Indication_treatment','indication_symptomatic_relief','contraindication', 'effect']
excel = 'Drug-disease relations annotations - Medical Expert Aggregated.xlsx'
for sheet in sheets:
    df = pd.read_excel(excel, sheet_name=sheet)
    df['sheet'] = sheet
    tables.append(df)

#### Integrate

In [3]:
merged_table = pd.DataFrame()
for table in tables:
    merged_table = merged_table.append(table, ignore_index=True, sort=False)

In [4]:
len(merged_table)

747

In [5]:
merged_table.tail(5)

,#,context,disease_name,drug_name,workers_answers,medical_expert1,medical_expert2,medical_expert3,do_id,drug_brand_name,drug_id,label_id,sheet
742,NaN,NaN,NaN,NaN,Effect,NaN,NaN,NaN,NaN,NaN,NaN,NaN,effect
743,NaN,NaN,NaN,NaN,I dont know,NaN,NaN,NaN,NaN,NaN,NaN,NaN,effect
744,101.0,"Hypersensitivity to doxepin hydrochloride, ina...",GLAUCOMA,doxepin,Effect,effect,Contraindication,?,DOID_1686,Silenor,DB01142,../DailyMedExtracter/prescription/temp_xml/520...,effect
745,NaN,NaN,NaN,NaN,Effect,NaN,NaN,NaN,NaN,NaN,NaN,NaN,effect
746,NaN,NaN,NaN,NaN,Effect,NaN,NaN,NaN,NaN,NaN,NaN,NaN,effect


In [6]:
merged_table = merged_table.dropna(subset=['medical_expert1','medical_expert2','medical_expert3'])

---
## Gathering experts tables

In [7]:
labels = pd.read_csv('Expert2Relation_Mapping.csv',sep=',')

In [8]:
label_dict=labels.set_index('text').to_dict()

In [9]:
merged_table['medical_expert1'].replace(label_dict['relation'], inplace=True)

In [10]:
merged_table['medical_expert2'].replace(label_dict['relation'], inplace=True)

In [11]:
merged_table.medical_expert2= merged_table.medical_expert2.str.rstrip('\n')

In [12]:
merged_table['medical_expert3'].replace(label_dict['relation'], inplace=True)

In [13]:
merged_table.drop_duplicates(subset=['label_id','do_id','drug_id'],inplace=True)

In [14]:
merged_table = merged_table[merged_table.medical_expert3 != 'Duplicate']

In [15]:
expert_consensus=[]
inter_agree_experts =[]
for index, row in merged_table.iterrows():
    if row['medical_expert1'] == row['medical_expert2'] and row['medical_expert2'] == row['medical_expert3']:
        expert_answer = row['medical_expert1']
        inter_agree = 3
    elif row['medical_expert1'] == row['medical_expert2']:
        expert_answer = row['medical_expert1']
        inter_agree = 2
    elif row['medical_expert2'] == row['medical_expert3']:
        expert_answer = row['medical_expert2']
        inter_agree = 2
    elif row['medical_expert1'] == row['medical_expert3']:
        expert_answer = row['medical_expert1'] 
        inter_agree = 2
    else:
        expert_answer = 'No consensus'
        inter_agree = 0

    
    if expert_answer in ['Indication: Treatment','Contraindication', 'Indication: Symptomatic Relief','Effect']:
        expert_consensus.append(expert_answer)
        inter_agree_experts.append(inter_agree)
    else:
        expert_consensus.append('No consensus')
        inter_agree_experts.append(0)


In [16]:
len(merged_table)

359

In [17]:
merged_table['expert_consensus'] = expert_consensus

In [18]:
merged_table['inter_agree_experts'] =inter_agree_experts

## Create an expert consensus tableaxis=

In [19]:
merged_table.shape

(359, 15)

In [20]:
merged_table.to_csv('expert_table.csv',index=False)

In [21]:
merged_table.expert_consensus.value_counts()

Indication: Treatment             128
Contraindication                  107
Indication: Symptomatic Relief     49
Effect                             47
No consensus                       28
Name: expert_consensus, dtype: int64